In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [58]:
df = pd.read_csv('titanic/train.csv')
print(len(df))
df = df.dropna(subset=['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Ticket'])
print(len(df))
df.head()

891
712


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [59]:
sex_encoded = []
for ind, row in df.iterrows():
    if row['Sex'] == 'male':
        sex_encoded.append(1)
    elif row['Sex'] == 'female':
        sex_encoded.append(0)
df['SexEncoded'] = sex_encoded
df = df.drop(labels=['Sex', 'Cabin', 'Ticket', 'Name', 'Embarked'], axis=1)
df.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,SexEncoded
0,1,0,3,22.0,1,0,7.2500,1
1,2,1,1,38.0,1,0,71.2833,0
2,3,1,3,26.0,0,0,7.9250,0
3,4,1,1,35.0,1,0,53.1000,0
4,5,0,3,35.0,0,0,8.0500,1


In [72]:
y = df['Survived']
X = df.drop(['Survived'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
print(len(X_train), len(X_test))

605 107


In [73]:
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)
y_predicted = classifier.predict(X_test)
accuracy_score(y_test, y_predicted)

C:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.8037383177570093

In [93]:
def preproc_data(df):
    df = df.dropna(subset=['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Ticket'])
    sex_encoded = []
    for ind, row in df.iterrows():
        if row['Sex'] == 'male':
            sex_encoded.append(1)
        elif row['Sex'] == 'female':
            sex_encoded.append(0)
    df['SexEncoded'] = sex_encoded
    df = df.drop(labels=['Sex', 'Cabin', 'Ticket', 'Name', 'Embarked'], axis=1)
    return df

def random_forest_cross_val(df):
    y = df['Survived']
    X = df.drop(['Survived'], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_test)
    print('Accuracy cross-val:', accuracy_score(y_test, y_predicted))
    model.fit(X, y)
    return model

In [116]:
df = pd.read_csv('titanic/train.csv')
df = preproc_data(df)

model = random_forest_cross_val(df)
test_df = pd.read_csv('titanic/test.csv')
sex_encoded = []
for ind, row in test_df.iterrows():
    if row['Sex'] == 'male':
        sex_encoded.append(1)
    elif row['Sex'] == 'female':
        sex_encoded.append(0)
test_df['SexEncoded'] = sex_encoded
pass_id = test_df['PassengerId']
test_df = test_df.drop(labels=['Sex', 'Cabin', 'Ticket', 'Name', 'Embarked'], axis=1)
y_predict = []
for ind, row in test_df.iterrows():
    try:
        y_pr = model.predict(row.values.reshape(1, -1))[0]
    except:
        y_pr = 0
    y_predict.append(y_pr)
    
answer = pd.DataFrame(data={'PassengerId': pass_id.values, 'Survived': y_predict})
answer.head(20)
answer.to_csv('answer.csv', index=False)

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy cross-val: 0.8130841121495327
